In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import scipy.integrate as integrate
import sys
sys.path.append('../..')
from laplace_coeff_Thomas import *
from datanew import *
# needed for the laplace coefficients from the lookup table
cfs=LaplaceCoeffs()
dcfs=LaplaceCoeffsDerivatives()

In [2]:
# Konstanten Einheitenlos
M_STAR=1
MU1=3e-3
MU2=1e-3
M1=MU1*M_STAR
M2=MU2*M_STAR
mmr=2
G=1
R_0=1 #1 au

SIGMA_0=1e-3
SIGMASLOPE=1.5

ALPHAVISCOSITY=1e-5       
TSLOPE=0.5           
FLARINGINDEX=0.25
H_0=0.06
SIGMARATIO=0.3   #for kanagawa

R_IN=np.arange(1.1,3 - 0.002,0.001)
R_OUT=R_IN *mmr**(2/3)

In [3]:
# Functions for the Torque of ward8
def omega_kepler(r):
    return np.sqrt(G*M_STAR/(r**3))

def surface_density(r):
    return SIGMA_0* (r/R_0)**(-SIGMASLOPE)

def aspect_ratio(r):
    return H_0 *(r/R_0)**FLARINGINDEX

def xi(r,m):
    return m*aspect_ratio(r)

def vk(r):
    return np.sqrt(G*M_STAR/r)

def cs(r):
    return aspect_ratio(r)*vk(r)

def press(r):
    return surface_density(r)*cs(r)**2

def dfdlnr(f,r,*args):
    rp       = r * (1+1e-5)
    lnr      = np.log(r)
    lnrp     = np.log(rp)
    return (f(rp,*args)-f(r,*args))/(lnrp-lnr)
                                      
def dlnpdlnr(r):
    return dfdlnr(press,r)/press(r)
                                     

def omphi(r):
    return omega_kepler(r) * np.sqrt(1+(cs(r)**2/vk(r)**2)*dlnpdlnr(r))
                                     
def kappa(r):
    qkap = -dfdlnr(omphi,r)/omphi(r)
    return np.sqrt(2*(2-qkap))*omphi(r)


def Dstar(r,r0,m):
    return kappa(r)**2-m**2*(omphi(r)-omega_kepler(r0))**2+(m*cs(r)/r)**2

#cutoff radius
def r_lim_k(r):
    return r-aspect_ratio(r)*r *2/3 ,r+aspect_ratio(r)*r *2/3

In [4]:
#width of the gap

def deltain(r,mu,Sigmaratio):
    b= (0.5*Sigmaratio+0.16)
    
    c=(mu**2)
    c*=(aspect_ratio(r)) **(-3)
    c*=ALPHAVISCOSITY**(-1)

    return r*b*c**(0.25) 

In [5]:
#load the locations of D_p_star
Dstar10_in=np.loadtxt(f"/home/kim/Dokumente/Uni/Bachelor/Migration/Neuer_Start/D_p_star_in{mmr}_{H_0}_{FLARINGINDEX}_{SIGMA_0}.dat").reshape(len(np.arange(1,20)),len(np.arange(1.1,3 - 0.002,0.001)))
Dstar10_out=np.loadtxt(f"/home/kim/Dokumente/Uni/Bachelor/Migration/Neuer_Start/D_p_star_out{mmr}_{H_0}_{FLARINGINDEX}_{SIGMA_0}.dat").reshape(len(np.arange(1,20)),len(np.arange(1.1,3 - 0.002,0.001)))


Dstar10_in=Dstar10_in.T
Dstar10_out=Dstar10_out.T

In [6]:
# Matrix of Lindblad locations with the consideration of the gap width and the cutoff radius
def r_Llist(m,rp):
    
    rvin = Dstar10_in
    rvin=np.where(rvin==0, np.nan, rvin)
    rvout = Dstar10_out
    
    r1, r2 = r_lim_k(rp)
    r1, r2 = r1[:,None], r2[:,None]
    
    delta_in = deltain(rp, MU1, SIGMARATIO)/2
    delta_out = deltain(rp, MU2, SIGMARATIO)/2
   
    finalin = np.where(rvin  < (rp - delta_in)[:,None], rvin, np.nan)
    finalin =  np.where(finalin != np.nan,np.where(finalin >= r1, r1, finalin),np.nan)
    
    finalout= np.where(rvout > (rp + delta_out)[:,None], rvout, np.nan)
    finalout= np.where(finalout != np.nan,np.where(finalout <= r2, r2, finalout), np.nan)
        
    return finalin, finalout

In [7]:
# Value of Lindblad locations with the consideration of the gap width and the cutoff radius
def r_Lvalue(m,rp_index,rp):
    
    rvin = Dstar10_in[rp_index][m-1]
    rvin=np.where(rvin==0, np.nan, rvin)
    rvout = Dstar10_out[rp_index][m-1]
    
    r1, r2 = r_lim_k(rp)
    
    delta_in = deltain(rp, MU1, SIGMARATIO)/2
    delta_out = deltain(rp, MU2, SIGMARATIO)/2
   
    finalin = np.where(rvin  < (rp - delta_in), rvin, np.nan)
    finalin =  np.where(finalin != np.nan,np.where(finalin >= r1, r1, finalin),np.nan)
    
    finalout= np.where(rvout > (rp + delta_out), rvout, np.nan)
    finalout= np.where(finalout != np.nan,np.where(finalout <= r2, r2, finalout), np.nan)
        
    return finalin, finalout


In [9]:
# Functions for the Torque of ward8
def Psi(m,r,rp):

    laplace = np.zeros((len(rp),len(m)))
    dlaplace= np.zeros((len(rp),len(m)))
        
    for i in np.arange(len(m)):
        for j in np.arange(len(rp)):
            laplace[j][i] = cfs( r[j][i]/rp[j], m[i])
            dlaplace[j][i] = dcfs(r[j][i]/rp[j], m[i])
            
    u = np.pi/2 
    v = 1/m *np.absolute(dlaplace)

    l = 2* np.sqrt(1+xi(r,m)**2)
    l *= laplace
    

    rs = u*(v+l)
    return rs

def Chi(m, epsilon,mu , r,rp):

    rv= epsilon*4/3
    rv*= mu**2 
    rv*=surface_density(r)    
    ah=m**2 *(omega_kepler(rp)**2)[:,None]
    t=(r/rp[:,None])**(3/2) *rp[:,None]**4

    
    return rv*ah*t

def Q(m, epsilon,r,rp):
    
    a=epsilon*(1-TSLOPE)*aspect_ratio(r)
    
    a*=1/(3* (r/rp[:,None])**(3/2))
    a*=xi(r,m)/np.sqrt(1+xi(r,m)**2) 
    rt=1/(1- a)
    return rt



def Gamma(m,r):
    
    y=np.sqrt(1+xi(r,m)**2)
    y*=(1+4*xi(r,m)**2)

    ry=1/y
    return ry

In [13]:
#Torque of ward8
def T(m,mu,rp,h_0):

    rvin=r_Llist(m,rp)[0]
    rvout=r_Llist(m,rp)[1]

    T_in=(-1)*Chi(m, -1,mu ,rvin,rp) *Q(m, -1,rvin,rp) *Psi(m,rvin,rp)**2 *Gamma(m,rvin)
    T_out=(-1)*Chi(m, 1,mu ,rvout,rp) *Q(m, 1,rvout,rp) *Psi(m,rvout,rp)**2 *Gamma(m,rvout)
    
    T_in=np.nan_to_num(T_in)
    T_out=np.nan_to_num(T_out)

    return T_in, T_out



In [14]:
#Thomas section wise kanagawa profile
def Kanagawa17Profile(r, rp, Mp, ALPHAVISCOSITY):
    """ Gap profile from Kanagawa et al. 2017 Eq. (6).
    
    This function calculates the gap shape Sigma(r)/Sigma0.8
    Parameters
    ----------
    r : array
        Grid radii.
    rp : float
        Orbital radius of the planet.
    Mp : float
        Planet mass in units of stellar mass.
    h : float
        Aspect ratio (H/R) at the location of the planet.
    alpha : float
        Viscous alpha parameter.
        
    Returns
    -------
    array
        Reduction factor of the surface density.
    
    """
    Sigma = np.ones_like(r)
    
    h=aspect_ratio(r)
    Kp = Mp**2 * h**(-3) * ALPHAVISCOSITY**(-1) # Eq. (2)
    Kp14 = Kp**(1/4)
    K = Mp**2 * h**(-5) *  ALPHAVISCOSITY**(-1) # Eq. (11)
    
    SigmaMin = 1 / (1+0.04*K) # Eq. (10)
    
    DR1 = (SigmaMin/4 + 0.08)*Kp14*rp # Eq. (8)
    DR2 = 0.33*Kp14*rp # Eq. (9)
    
    # regions according to Eq. (6)
    Dr = np.abs(r - rp)
    region1 = Dr <= DR1
    region2 = np.logical_and(Dr > DR1, Dr < DR2)
    
    # surface denity ratio according to Eq. (6)
    Sigma[region1] = SigmaMin*0.4
    Sigma[region2] = 4/Kp14 * Dr[region2]/rp - 0.32
    
    # The third region for Dr > DR2 is just Sigma0, thus it stays 1 here.
    
    return Sigma

In [15]:
#Inner Torques with the weigtening of the Kanagwa profile
def Torque_in_all(m_max):
    torque_in=[]
    
    m=np.arange(1,m_max,1)
    
    Kanagawa_in=[[Kanagawa17Profile(r_Lvalue(i,j, R_IN[j])[0], R_IN[j], M1,ALPHAVISCOSITY)  for i in m]for j in np.arange(0,len(R_IN))]
    T_in=T(m,MU1,R_IN,H_0)[0]
    torque_in = np.sum(T_in*Kanagawa_in, axis=1)
    
    return torque_in

In [16]:
#Outer Torques with the weigtening of the Kanagwa profile
def Torque_out_all(m_max):
    torque_out=[]
    m_k=np.arange(1,m_max,1)
    
    Kanagawa_out=[[Kanagawa17Profile(r_Lvalue(i,j,R_OUT[j])[1], R_OUT[j], M2, ALPHAVISCOSITY) for i in m_k]for j in np.arange(0,len(R_OUT))] 
    T_out=T(m_k,MU2,R_OUT,H_0)[1]
    torque_out = np.sum(T_out*Kanagawa_out,axis=1)

    return torque_out


# mit Gewichtung

In [19]:
T20in=Torque_in_all(20)
T20out=Torque_out_all(20)

ValueError: A value in x_new is above the interpolation range.

In [ ]:
#plot the torques
plt.figure(figsize=(10,5))
plt.plot(R_IN,T20in,label='all inner Torques')
plt.plot(R_OUT,T20out,label='all outer Torques')


plt.xlabel('r',fontsize=13)
plt.ylabel('Torques',fontsize=13)
plt.title('all Torques',fontsize=13)
plt.axhline(0, c = "k", linestyle = "--")
plt.legend(fontsize=13)

# Migrationsraten

In [ ]:
#Migration rate which results of the angular momentum calculation

#hier mp'= M1+M2
def rate(T,a,M1,M2):
    Sigma= np.sqrt(G*M_STAR/a**3)
    
    rate=2*T/(Sigma*a*(M1+M2))
    return rate

In [ ]:
#Sum over the rates to plot them
T_allgew=[sum(x) for x in zip(Torque_in_all(20),Torque_out_all(20))]

rate_ingew= [rate(T,r,M1,M2) for T,r in zip(Torque_in_all(20),R_IN)]
rate_outgew= [rate(T,r,M1,M2) for T,r in zip(Torque_out_all(20),R_OUT)]
rate_allgew=[rate(T,r,M1,M2) for T,r in zip(T_allgew,R_IN)]

In [ ]:
#Plot the rates
plt.figure(figsize=(15,7))
plt.plot(R_IN,rate_ingew  , label='Migration rate inner Torques')
plt.plot(R_OUT,rate_outgew, label='Migration rate outer Torques')
plt.plot(R_IN,rate_allgew, label='Migration rate insgesamt')

plt.xlabel('r',fontsize=13)
plt.ylabel('Migration rate',fontsize=13)
plt.legend(fontsize=13)
plt.axhline(0, c = "k", linestyle = "--")
plt.grid()
plt.minorticks_on()

#plt.savefig(f'Ward8_D_p_star_pictures/rates_ward8_D_p_star_{FLARINGINDEX}_{SIGMA_0}_{ALPHAVISCOSITY}_{M1}_{SIGMARATIO}_{H_0}.png')

# Abspeichern der Daten

In [ ]:
orte=BM08newortederrate
index=[int(np.round((i-1.1)/0.001)) for i in orte]
    

file2write=open(f"BM08Ward8_D_p_star_{mmr}_{SIGMARATIO}.txt",'a+')
file2write.write('newtheory_h0_006_alpha_1e_5_planets_3_1_sigma0_1e_3'+' ' )

file2write.write(str(rate_allgew[index[9]])+' ')


file2write.write(str(SIGMARATIO)+' ')
file2write.write(str(M1)+' ')
file2write.write(str(FLARINGINDEX)+' ')
file2write.write(str(R_IN[index[9]])+' ')
    
file2write.write('\n')
file2write.close()